In [2]:
import numpy as np
import os
import sklearn.compose
import sklearn.datasets
import sklearn.model_selection
import sklearn.pipeline
import sklearn.preprocessing
import sklearn.linear_model
import sklearn.metrics

In [8]:
x = []
dataset = np.load("rental_competition.train.npz")
for key, value in dataset.items():
    setattr(x, key, value)

AttributeError: 'list' object has no attribute 'target'

In [3]:
class Dataset:
    def __init__(self,
                    name="rental_competition.train.npz",
                    url="https://ufal.mff.cuni.cz/~straka/courses/npfl129/2223/datasets/"):
            if not os.path.exists(name):
                print("Downloading dataset {}...".format(name), file=sys.stderr)
                urllib.request.urlretrieve(url + name, filename=name)

            # Load the dataset and return the data and targets.
            dataset = np.load(name)
            print('in hiar')
            for key, value in dataset.items():

                setattr(self, key, value)

In [14]:
dataset = Dataset()

In [23]:
dataset.items()


AttributeError: 'Dataset' object has no attribute 'items'

In [24]:
dataset = np.load("rental_competition.train.npz")

In [86]:
train.data

array([[ 3.    ,  1.    ,  6.    , ...,  0.697 ,  0.27  ,  0.194 ],
       [ 1.    ,  1.    ,  1.    , ...,  0.2273,  0.41  ,  0.2239],
       [ 4.    ,  0.    , 10.    , ...,  0.303 ,  0.66  ,  0.2836],
       ...,
       [ 4.    ,  1.    , 12.    , ...,  0.4848,  0.59  ,  0.2836],
       [ 1.    ,  0.    ,  1.    , ...,  0.1515,  0.86  ,  0.2239],
       [ 3.    ,  1.    ,  9.    , ...,  0.5909,  0.89  ,  0.    ]])

In [31]:
train = Dataset()

in hiar


In [37]:
print(train.key)


AttributeError: 'Dataset' object has no attribute 'key'

In [64]:
train.data[:, 8]
x = [8*True]
x.append([4*False])


In [67]:
col_int = np.all(train.data.astype(int) == train.data, axis=0)


In [68]:
col_int

array([ True,  True,  True,  True,  True,  True,  True,  True, False,
       False, False, False])

In [73]:
transformer = sklearn.compose.ColumnTransformer([("Cat", sklearn.preprocessing.OneHotEncoder(sparse=False, handle_unknown="ignore"), col_int), ("Std", sklearn.preprocessing.StandardScaler(), ~col_int)])

In [81]:
polynomial = sklearn.preprocessing.PolynomialFeatures(2, include_bias=False)
pipeline = sklearn.pipeline.Pipeline([("transformer", transformer), ("polynomial", polynomial)])

In [75]:
pipeline

Pipeline(steps=[('transformer',
                 ColumnTransformer(transformers=[('Cat',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False),
                                                  array([ True,  True,  True,  True,  True,  True,  True,  True, False,
       False, False, False])),
                                                 ('Std', StandardScaler(),
                                                  array([False, False, False, False, False, False, False, False,  True,
        True,  True,  True]))])),
                ('polynomial', PolynomialFeatures(include_bias=False))])

In [29]:
def pipeline(train, test, d):
    col_int = np.all(train.astype(int) == train, axis=0)
    transformer = sklearn.compose.ColumnTransformer([("Cat", sklearn.preprocessing.OneHotEncoder(sparse=False, handle_unknown="ignore"), col_int), ("Std", sklearn.preprocessing.StandardScaler(), ~col_int)])
    polynomial = sklearn.preprocessing.PolynomialFeatures(d, include_bias=False)
    pipeline = sklearn.pipeline.Pipeline([("transformer", transformer), ("polynomial", polynomial)])    
    train_data = pipeline.fit_transform(train)
    test_data = pipeline.transform(test)

    return train_data, test_data
    

In [36]:
alpha = np.ones((len(X2), 1))
X2 = np.concatenate((X2, alpha), axis=1)  

In [42]:
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(train.data, train.target, test_size=0.2, random_state=42)

In [43]:
X, X2 = pipeline(X_train, X_test, 3)

In [153]:
X.shape

(800, 61)

In [35]:
model = sklearn.linear_model.LinearRegression().fit(X, y_train)

In [19]:
X_train.shape

(950, 12)

In [55]:
pred = model.predict(X2)

In [56]:
rmse = sklearn.metrics.mean_squared_error(y_test, pred, squared=False)
rmse

69.59408127832836

In [54]:
model = sklearn.linear_model.Ridge(10).fit(X, y_train)

In [225]:
lambdas = np.geomspace(0.01, 10, num=500)
rmses = [0 for i in range(len(lambdas))]
for count, value in enumerate(lambdas):
    model = sklearn.linear_model.Ridge(value).fit(X, y_train)
    pred = model.predict(X2)
    rmses[count] = sklearn.metrics.mean_squared_error(y_val, pred, squared=False)

In [226]:
best_rmse = min(rmses)
best_rmse

80.11287012382681

In [227]:
best_lambda = lambdas[rmses.index(best_rmse)]
best_lambda


0.01

In [223]:
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(train.data, train.target, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = sklearn.model_selection.train_test_split(X_train, y_train, test_size=0.25, random_state=42)

In [200]:
generator = np.random.RandomState(42)

In [203]:
weights = generator.uniform(size=X.shape[1], low=-0.1, high=0.1)

train_rmses, test_rmses = [], []
for epoch in range(50):
    permutation = generator.permutation(X.shape[0])

    for i in range(X.shape[0] // 100):
        data_b = X[permutation[i * 100 : (i + 1) * 100]]
        target_b = y_test[permutation[i * 100 : (i + 1) * 100]]
        gradient = 1/abs(100)*(np.matmul((np.matmul(data_b, weights) - target_b), data_b))
        weights = weights - 0.01 * (gradient + 0 * weights)

    # TODO: Process the data in the order of `permutation`. For every
    # `args.batch_size` of them, average their gradient, and update the weights.
    # A gradient for example `(x_i, t_i)` is `(x_i^T weights - t_i) * x_i`,
    # and the SGD update is
    #   weights = weights - args.learning_rate * (gradient + args.l2 * weights)`.
    # You can assume that `args.batch_size` exactly divides `train_data.shape[0]`.


    # TODO: Append current RMSE on train/test to `train_rmses`/`test_rmses`.
    pred_tr = np.zeros(len(X))
    for i in range(len(X)):
        pred_tr[i] = sum(weights*X[i])

    pred_te = np.zeros(len(X2))
    for i in range(len(X2)):
        pred_te[i] = sum(weights*X2[i])

    train_rmses.append(np.sqrt((sum((pred_tr-train_target)**2))/len(pred_tr)))
    test_rmses.append(np.sqrt((sum((pred_te-test_target)**2))/len(pred_te)))
# TODO: Compute into `explicit_rmse` test data RMSE when fitting
# `sklearn.linear_model.LinearRegression` on `train_data` (ignoring `args.l2`)

IndexError: index 489 is out of bounds for axis 0 with size 200

In [204]:
from sklearn.linear_model import SGDClassifier

In [10]:
for i in range(1, 4):
            X, X2 = pipeline(X_train, X_test, i)


            lambdas = np.geomspace(0.01, 10, num=500)
            rmses = [0 for i in range(len(lambdas))]
            for count, value in enumerate(lambdas):
                model = sklearn.linear_model.Ridge(value).fit(X, y_train)
                pred = model.predict(X2)
                rmses[count] = sklearn.metrics.mean_squared_error(y_test, pred, squared=False)
            
            best_rmse = min(rmses)
            best_rmse

            best_lambda = lambdas[rmses.index(best_rmse)]
            best_lambda

            print(best_lambda)

            model = sklearn.linear_model.Ridge(best_lambda).fit(train.data, train.target)

0.39191002494334043
3.9554362447347016


KeyboardInterrupt: 